In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../owl-wms')
sys.path.append('../owl-vaes')
sys.path.append('..')

In [3]:
#from lat2lat.data import get_loader
from owl_vaes.data.local_cod import get_loader
import torch
from lat2lat.models import WANDCAEPair

In [4]:
batch_size = 5

In [5]:
loader = get_loader(batch_size, root='/mnt/nas/YoutubeCoD/')

In [6]:
d = next(iter(loader))
print(d.shape)

torch.Size([5, 3, 360, 640])


In [9]:
pair = WANDCAEPair()
pair.bfloat16()
pair.cuda()

print(d.shape)
x_wan, x_dcae = pair.preprocess(d.unsqueeze(0).float().cuda())
mean_wan, mean_dcae = pair(x_wan, x_dcae)
print(mean_wan.shape, mean_dcae.shape)

torch.Size([5, 3, 360, 640])
torch.Size([1, 5, 3, 360, 640])
torch.Size([1, 2, 16, 64, 64]) torch.Size([1, 5, 128, 4, 4])


In [10]:
import tarfile
import io
import os

# Create data directory if it doesn't exist
os.makedirs('../../data', exist_ok=True)

# Create tar file in memory
tar_buffer = io.BytesIO()

with tarfile.open(fileobj=tar_buffer, mode='w') as tar:
    # Add mean_wan tensor
    wan_buffer = io.BytesIO()
    torch.save(mean_wan, wan_buffer)
    wan_buffer.seek(0)
    wan_info = tarfile.TarInfo(name='mean_wan.pt')
    wan_info.size = len(wan_buffer.getvalue())
    tar.addfile(wan_info, wan_buffer)
    
    # Add mean_dcae tensor
    dcae_buffer = io.BytesIO()
    torch.save(mean_dcae, dcae_buffer)
    dcae_buffer.seek(0)
    dcae_info = tarfile.TarInfo(name='mean_dcae.pt')
    dcae_info.size = len(dcae_buffer.getvalue())
    tar.addfile(dcae_info, dcae_buffer)

# Print the size of the tar buffer
tar_buffer.seek(0)
tar_size = len(tar_buffer.getvalue())
print(f"Tar buffer size: {tar_size} bytes ({tar_size / 1024 / 1024:.2f} MB)")

# Write the tar file to disk
tar_buffer.seek(0)
with open('../../../data/pair.tar', 'wb') as f:
    f.write(tar_buffer.getvalue())

print("Tensors saved to ../../../data/pair.tar")

Tar buffer size: 552960 bytes (0.53 MB)
Tensors saved to ../../data/pair.tar
